In [ ]:
### Flynn's Taxonomy 

Characterize machines by number of instruction streams and data streams
  * Defined in 1972.  Still common practice.
  * A little too restrictive, but a starting place

### SISD: single instruction, single data

Perform  a stream on instructions on a stream of data
  * The von Neumann architecture
  * Conforms to serial algorithmic analysis

### SIMD: single instruction, multiple data

* Single control stream
  * All processors execute the same instruction at the same time
  * against different data
  * synchronous execution or 'in lock step'
* Fine-grained parallelism without inter-process communication
* Limited examples  
  * Intel vector processors
  * GPU stream processor

The examples are limited because 

### MISD: multiple instruction, single data
Irrelevant.  No such machines.

### MIMD: multiple instruction, multiple data


